In [4]:
# S2 signal script
import sys
sys.path.append('/home/investigator/mariandbt/python/notebooks/modules')

from import_modules import *

import set_up as setup
import s2_signal as s2sig


In [267]:
n_ie = int(1e4)

In [268]:
np.random.poisson(100, n_ie)

array([110,  88,  95, ..., 101, 109, 102])

In [269]:
list_of_x = []
list_of_y = []
list_of_q = []

x_min = 0
x_max = 490
y_min = -85
y_max = 85

for i in range(100):
    x_pos = np.random.uniform(x_min, x_max, n_ie)
    y_pos = np.random.uniform(y_min, y_max, n_ie)
    charge = np.random.poisson(100, n_ie)
    
    list_of_x.append(x_pos)
    list_of_y.append(y_pos)
    list_of_q.append(charge)



In [270]:
bin_width = 10 # [mm]

x_nbins = int((x_max - x_min)/bin_width)
y_nbins = int((y_max - y_min)/bin_width)
# x_nbins = y_nbins = n_bins
# bins = (x_nbins, y_nbins)

xedges = np.linspace(x_min, x_max, x_nbins + 1)
yedges = np.linspace(y_min, y_max, y_nbins + 1)

In [271]:
for ii in range(len(list_of_x)):
    
    (x_pos, y_pos, charge) = (list_of_x[ii], list_of_y[ii], list_of_q[ii])
    print(x_pos[0], y_pos[0], charge[0])

130.00653693921402 3.0579344143033893 73
482.0700328147747 -38.61186613381129 96
359.5540685224407 -84.97670944541866 102
235.86720386869692 52.41630387445602 80
48.48161355823981 -62.71460221946143 98
255.16780119292437 -25.688365891105512 92
42.187708141171456 62.2073094795131 93
233.28687102766204 -32.752669785088145 106
411.3917704367448 72.43062773325104 97
144.03395108393545 31.943094161267354 95
480.0202260693155 -21.93966183441254 81
130.26072108896332 -26.49313803507591 85
179.20740100281265 0.9769581314154294 96
250.30435966493985 25.042903112726307 92
475.91729611081166 12.589064355673088 107
411.68422121169874 -48.760913318692694 91
396.5802636683658 -21.823725923322122 113
342.42359063110626 13.387677648029594 98
210.49222287742327 -4.8260918804847535 99
417.3980815827931 29.306895328196433 94
240.8420907776665 40.08028724141765 122
462.3429829876077 -42.91460036326836 83
115.19991948929992 25.187056709351495 107
460.5805624842268 -49.58179558219342 91
281.12660505445484 7

In [320]:

# Initialize lists to store statistics for each bin
charge_per_bin_hist = np.zeros((x_nbins, y_nbins))
counts_per_bin_hist = np.zeros((x_nbins, y_nbins))
    
# ****************************************************************************************

for ii in range(len(list_of_x)):
    
    (x_pos, y_pos, charge) = (list_of_x[ii], list_of_y[ii], list_of_q[ii])
    
    # Create a 2D histogram
    hist, _, _ = np.histogram2d(x_pos, y_pos,
                                bins=(xedges, yedges),
                                weights = charge,
                                density=False);
    charge_per_bin_hist = charge_per_bin_hist + hist

    hist_counts, _, _ = np.histogram2d(x_pos, y_pos,
                                       bins=(xedges, yedges),
                                       density=False);
    counts_per_bin_hist = counts_per_bin_hist + hist_counts

# ****************************************************************************************

# mean_per_bin_hist = np.where(counts_per_bin_hist > 0., charge_per_bin_hist / counts_per_bin_hist, 0.);
mean_per_bin_hist = np.divide(charge_per_bin_hist, counts_per_bin_hist, 
                              out=np.zeros_like(charge_per_bin_hist), 
                              where=counts_per_bin_hist != 0)


In [321]:
charge_per_bin_hist

array([[120604., 119384., 121672., 122593., 122594., 118309., 126351.,
        118358., 121164., 115412., 119253., 118045., 116800., 120007.,
        122319., 122252., 119286.],
       [121847., 119286., 117489., 113887., 121370., 117379., 116727.,
        118842., 113147., 121743., 119544., 121069., 117582., 119562.,
        117716., 123871., 120488.],
       [125063., 124193., 115853., 114700., 121292., 114079., 118744.,
        115957., 127666., 124689., 122484., 114248., 117271., 118177.,
        120369., 120485., 121090.],
       [121901., 118253., 120361., 118299., 118884., 120472., 117792.,
        119836., 117763., 121406., 117874., 117974., 121225., 124313.,
        118839., 121741., 123618.],
       [122766., 118505., 120427., 118423., 117260., 127033., 126069.,
        127614., 123687., 121754., 119617., 123787., 115827., 118289.,
        117443., 121952., 118607.],
       [117972., 114325., 122072., 119254., 122736., 124439., 121084.,
        122630., 113323., 125823., 1219

In [322]:
i = 3 
j = 7

In [323]:
aa = np.divide(charge_per_bin_hist, counts_per_bin_hist,
               out=np.zeros_like(charge_per_bin_hist), where=counts_per_bin_hist != 0)[i][j];

bb = charge_per_bin_loop[i][j]/counts_per_bin_loop[i][j] if counts_per_bin_loop[i][j] > 0 else 0
cc = charge_per_bin_hist[i][j]/counts_per_bin_hist[i][j]

print('aa = ', aa)
print('bb = ', bb)
print('cc = ', cc)
print('mean_per_bin_hist[i][j] = ', mean_per_bin_hist[i][j])
print('charge_per_bin_hist[i][j] = ', charge_per_bin_hist[i][j])
print('counts_per_bin_hist[i][j] = ', counts_per_bin_hist[i][j])

aa =  99.86333333333333
bb =  99.86333333333333
cc =  99.86333333333333
mean_per_bin_hist[i][j] =  99.86333333333333
charge_per_bin_hist[i][j] =  119836.0
counts_per_bin_hist[i][j] =  1200.0


In [324]:
x_pos, y_pos, charge

(array([114.72046908, 334.45828986,  98.45024207, ...,  31.39117067,
        463.98224321,  46.39880357]),
 array([-66.67551361,  19.99650314,   0.7226142 , ..., -32.29825956,
        -29.93379661,  24.94895541]),
 array([102,  88, 104, ..., 113, 123,  95]))

In [325]:
xedges, yedges

(array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
        110., 120., 130., 140., 150., 160., 170., 180., 190., 200., 210.,
        220., 230., 240., 250., 260., 270., 280., 290., 300., 310., 320.,
        330., 340., 350., 360., 370., 380., 390., 400., 410., 420., 430.,
        440., 450., 460., 470., 480., 490.]),
 array([-85., -75., -65., -55., -45., -35., -25., -15.,  -5.,   5.,  15.,
         25.,  35.,  45.,  55.,  65.,  75.,  85.]))

In [326]:
xedges[i], xedges[i + 1], yedges[j], yedges[j + 1]

(30.0, 40.0, -15.0, -5.0)

In [327]:
# Indices of data points in the current bin
mask = ((x_pos >= xedges[i]) & (x_pos < xedges[i + 1]) & 
        (y_pos >= yedges[j]) & (y_pos < yedges[j + 1]))

# Extract values and weights in the current bin
charge[mask]

array([116, 100,  98,  91, 109, 104, 101,  98, 119, 111, 111, 118,  90,
       112])

In [328]:
# ****************************************************************************************
# Initialize lists to store statistics for each bin
mean_per_bin = np.zeros((x_nbins, y_nbins))
charge_per_bin_loop = np.zeros((x_nbins, y_nbins))
counts_per_bin_loop = np.zeros((x_nbins, y_nbins))

for ii in range(len(list_of_x)):
    
    (x_pos, y_pos, charge) = (list_of_x[ii], list_of_y[ii], list_of_q[ii])

    # Iterate over each bin
    for i in range(x_nbins):
        for j in range(y_nbins):
            # Indices of data points in the current bin
            mask = ((x_pos >= xedges[i]) & (x_pos < xedges[i + 1]) & 
                    (y_pos >= yedges[j]) & (y_pos < yedges[j + 1]))

            # Extract values and weights in the current bin
            values_in_bin = charge[mask]

            # Calculate weighted mean and standard deviation
#             mean_value = np.mean(values_in_bin)

#             if ((i == 3) & (j == 7)):
#                 print('ii = ', ii)
#                 print('values_in_bin.sum() = ', values_in_bin.sum())
#                 print('charge_per_bin_loop[i][j] = ', charge_per_bin_loop[i][j])
            # Append to lists
            charge_per_bin_loop[i][j] = charge_per_bin_loop[i][j] + values_in_bin.sum()
            counts_per_bin_loop[i][j] = counts_per_bin_loop[i][j] + len(values_in_bin)
            
#             if ((i == 3) & (j == 7)):
#                 print('ii = ', ii)
#                 print('values_in_bin.sum() = ', values_in_bin.sum())
#                 print('charge_per_bin_loop[i][j] = ', charge_per_bin_loop[i][j])
#                 print('counts_per_bin_loop[i][j] = ', counts_per_bin_loop[i][j])
            
for i in range(x_nbins):
    for j in range(y_nbins):
        
        mean_per_bin[i][j] = charge_per_bin_loop[i][j]/counts_per_bin_loop[i][j]
#         mean_per_bin[i][j] = charge_per_bin_loop[i][j] / counts_per_bin_loop[i][j] if counts_per_bin_loop[i][j] > 0 else 0


#         if ((i == 3) & (j == 7)):
#             print('mean_per_bin[i][j] = ', mean_per_bin[i][j])
#             print('charge_per_bin_loop[i][j] = ', charge_per_bin_loop[i][j])
#             print('counts_per_bin_loop[i][j] = ', counts_per_bin_loop[i][j])
            
mean_per_bin_loop = np.nan_to_num(mean_per_bin)
# ****************************************************************************************

In [329]:
mean_per_bin_hist.T

array([[100.00331675,  99.87459016,  99.65179283,  99.75531915,
        100.13539967, 100.14601019, 100.15364807,  99.87680544,
         99.75180723,  99.86729452,  99.82094595, 100.07058824,
         99.95488105, 100.16134454, 100.05666401, 100.06899286,
         99.54530341,  99.92121212, 100.20996732,  99.56949153,
        100.26567657, 100.18270008, 100.55953372,  99.98795181,
        100.32656514, 100.13755274,  99.87183811,  99.52737995,
         99.61097046,  99.60854987, 100.44049587, 100.1799507 ,
        100.45693191, 100.01558655,  99.90748528, 100.37323358,
         99.81544715, 100.08463435, 100.22670025, 100.39630563,
        100.27542373,  99.36833192, 100.2096635 ,  99.9747262 ,
         99.85575365,  99.48379254,  99.7061293 ,  99.55520254,
        100.32512315],
       [100.23845508,  99.8209205 ,  99.75341365, 100.04483926,
        100.4279661 , 100.54969217, 100.10442774,  99.90206186,
        100.00080906, 100.1245196 ,  99.60346249,  99.78487395,
        100.08879

In [330]:
mean_per_bin_loop.T

array([[100.00331675,  99.87459016,  99.65179283,  99.75531915,
        100.13539967, 100.14601019, 100.15364807,  99.87680544,
         99.75180723,  99.86729452,  99.82094595, 100.07058824,
         99.95488105, 100.16134454, 100.05666401, 100.06899286,
         99.54530341,  99.92121212, 100.20996732,  99.56949153,
        100.26567657, 100.18270008, 100.55953372,  99.98795181,
        100.32656514, 100.13755274,  99.87183811,  99.52737995,
         99.61097046,  99.60854987, 100.44049587, 100.1799507 ,
        100.45693191, 100.01558655,  99.90748528, 100.37323358,
         99.81544715, 100.08463435, 100.22670025, 100.39630563,
        100.27542373,  99.36833192, 100.2096635 ,  99.9747262 ,
         99.85575365,  99.48379254,  99.7061293 ,  99.55520254,
        100.32512315],
       [100.23845508,  99.8209205 ,  99.75341365, 100.04483926,
        100.4279661 , 100.54969217, 100.10442774,  99.90206186,
        100.00080906, 100.1245196 ,  99.60346249,  99.78487395,
        100.08879

In [331]:
np.shape(mean_per_bin_hist)

(49, 17)

In [332]:
np.shape(mean_per_bin_loop)

(49, 17)

In [333]:
mean_index = np.zeros((x_nbins, y_nbins))
for i in range(x_nbins):
    for j in range(y_nbins):
        
        mean_index[i][j] = 100*i + j

In [334]:
mean_index[mean_per_bin_hist != mean_per_bin_loop]

array([], dtype=float64)

In [335]:
mean_per_bin_hist[mean_per_bin_hist != mean_per_bin_loop]

array([], dtype=float64)

In [336]:
mean_per_bin_loop[mean_per_bin_hist != mean_per_bin_loop]

array([], dtype=float64)

In [337]:
mean_per_bin_hist[3][7], mean_per_bin_hist[48][10]

(99.86333333333333, 99.72344161545215)

In [338]:
mean_per_bin_loop[3][7], mean_per_bin_loop[48][10]

(99.86333333333333, 99.72344161545215)

In [339]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Sequence, Tuple
from antea.elec import tof_functions as wvf 
from scipy import signal
#import fit_library as ftl
%matplotlib nbagg
%reload_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'antea'

In [ ]:
list_of_x = []
list_of_y = []
list_of_q = []

x_min = 0
x_max = 490
y_min = -85
y_max = 85

for i in range(100):
    x_pos = np.random.uniform(x_min, x_max, n_ie)
    y_pos = np.random.uniform(y_min, y_max, n_ie)
    charge = np.random.poisson(100, n_ie)
    
    list_of_x.append(x_pos)
    list_of_y.append(y_pos)
    list_of_q.append(charge)
    
bin_width = 10 # [mm]

x_nbins = int((x_max - x_min)/bin_width)
y_nbins = int((y_max - y_min)/bin_width)
# x_nbins = y_nbins = n_bins
# bins = (x_nbins, y_nbins)

xedges = np.linspace(x_min, x_max, x_nbins + 1)
yedges = np.linspace(y_min, y_max, y_nbins + 1)

In [345]:
charge_per_bin = {}
counts_per_bin = {}

for kk in range(len(list_of_x)):
    
    (ev_x0, ev_y0, event_charge) = (list_of_x[kk], list_of_y[kk], list_of_q[kk])
    
    for ii, selected_sens in enumerate(['sens_200', 'sens_204']):

        if selected_sens not in charge_per_bin.keys():
            charge_per_bin[selected_sens] = np.zeros((x_nbins, y_nbins))
            counts_per_bin[selected_sens] = np.zeros((x_nbins, y_nbins))

        for i in range(x_nbins):
            for j in range(y_nbins):
                # Indices of data points in the current bin
                mask = ((ev_x0 >= xedges[i]) & (ev_x0 < xedges[i + 1]) &
                        (ev_y0 >= yedges[j]) & (ev_y0 < yedges[j + 1]))

                # Extract values and weights in the current bin
                values_in_bin = event_charge[mask]

                charge_per_bin[selected_sens][i][j] = charge_per_bin[selected_sens][i][j] + values_in_bin.sum()
                counts_per_bin[selected_sens][i][j] = counts_per_bin[selected_sens][i][j] + len(values_in_bin)



/home/investigator/miniconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/investigator/miniconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [346]:
charge_per_bin[selected_sens]

array([[120604., 119384., 121672., 122593., 122594., 118309., 126351.,
        118358., 121164., 115412., 119253., 118045., 116800., 120007.,
        122319., 122252., 119286.],
       [121847., 119286., 117489., 113887., 121370., 117379., 116727.,
        118842., 113147., 121743., 119544., 121069., 117582., 119562.,
        117716., 123871., 120488.],
       [125063., 124193., 115853., 114700., 121292., 114079., 118744.,
        115957., 127666., 124689., 122484., 114248., 117271., 118177.,
        120369., 120485., 121090.],
       [121901., 118253., 120361., 118299., 118884., 120472., 117792.,
        119836., 117763., 121406., 117874., 117974., 121225., 124313.,
        118839., 121741., 123618.],
       [122766., 118505., 120427., 118423., 117260., 127033., 126069.,
        127614., 123687., 121754., 119617., 123787., 115827., 118289.,
        117443., 121952., 118607.],
       [117972., 114325., 122072., 119254., 122736., 124439., 121084.,
        122630., 113323., 125823., 1219

In [349]:
aa = '22v2'

In [351]:
f'{aa}'

'22v2'